In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

In [12]:
def search_amazon(keyword):
    print("Searching for the product on Amazon...")
    driver.get('https://www.amazon.in')
    search_box = driver.find_element_by_id('twotabsearchtextbox')
    search_box.send_keys(keyword)
    search_box.submit()

    try:
        # Check if a message indicating no results is present
        no_results_message = driver.find_element(By.XPATH, "//span[contains(text(), 'No results for')]")
        print("Product not found on Amazon.")
        return None
    except NoSuchElementException:
        pass

    print("Search on Amazon completed.")

In [13]:
# Function to search for a product on Flipkart and check if it exists
def search_flipkart(keyword):
    print("Searching for the product on Flipkart...")
    driver.get('https://www.flipkart.com')

    try:
        # Use an explicit wait to wait for the close button to appear
        wait = WebDriverWait(driver, 10)
        close_popup = wait.until(EC.presence_of_element_located((By.XPATH, '//button[@class="_2KpZ6l _2doB4z"]')))
        close_popup.click()
    except TimeoutException:
        # If the pop-up doesn't appear, continue without closing it
        pass
    
    search_box = driver.find_element_by_name('q')
    search_box.send_keys(keyword)
    search_box.submit()

    try:
        # Check if a message indicating no results is present
        no_results_message = driver.find_element(By.XPATH, "//div[contains(text(), 'Sorry, no results found!')]")
        print("Product not found on Flipkart.")
        return None
    except NoSuchElementException:
        pass

    print("Search on Flipkart completed.")

In [14]:
# Function to search for a product on Croma and check if it exists
def search_croma(keyword):
    print("Searching for the product on Croma...")
    driver.get('https://www.croma.com')
    search_box = driver.find_element_by_id('searchV2')  # Use the actual ID 'searchV2'
    search_box.send_keys(keyword)
    
    # Simulate pressing the "Enter" key to trigger the search
    search_box.send_keys(Keys.ENTER)

    try:
        # Check if a message indicating no results is present
        no_results_message = driver.find_element(By.XPATH, "//div[contains(text(), 'No results found.')]")
        print("Product not found on Croma.")
        return None
    except NoSuchElementException:
        pass

    print("Search on Croma completed.")

In [15]:
# Function to extract the price from Amazon
def extract_amazon_price():
    print("Extracting the price from Amazon...")
    amazon_soup = BeautifulSoup(driver.page_source, 'html.parser')
    amazon_price = amazon_soup.find_all('span', {'class': 'a-price-whole'})
    if amazon_price:
        amazon_price = amazon_price[0].get_text()
    else:
        amazon_price = None
    print("Price extraction from Amazon completed.")
    return amazon_price

In [16]:
# Function to extract the price from Flipkart
def extract_flipkart_price():
    print("Extracting the price from Flipkart...")
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, '_30jeq3')))
        flipkart_soup = BeautifulSoup(driver.page_source, 'html.parser')
        flipkart_price = flipkart_soup.find('div', {'class': '_30jeq3'}).get_text()
        print("Price extraction from Flipkart completed.")
        return flipkart_price
    except TimeoutException:
        print("Price element not found on Flipkart.")
        return None

In [17]:
# Function to extract the price from Croma using XPath
def extract_croma_price():
    print("Extracting the price from Croma...")
    
    try:
        # Wait for the price element to become visible
        wait = WebDriverWait(driver, 10)
        croma_price_element = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/main/div[3]/div[1]/div[2]/div/div/div[3]/ul/li[1]/div/div[2]/div[3]/div[1]')))
        croma_price = croma_price_element.text.strip()
    except Exception as e:
        print(f"Error extracting Croma price: {str(e)}")
        croma_price = None

    print("Price extraction from Croma completed.")
    return croma_price


In [18]:
# Configure Selenium webdriver
driver = webdriver.Chrome(r'C:\Users\KIIT\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe') 
product_keyword = input("Enter a product name/keyword : ")

# Search for the product and extract price on amazon
search_amazon(product_keyword)
amazon_price = extract_amazon_price()

# Search for the product and extract price on flipkart
search_flipkart(product_keyword)
flipkart_price = extract_flipkart_price()

# Search for the product and extract price on croma
search_croma(product_keyword)
croma_price = extract_croma_price()

# Close the Selenium webdriver
driver.quit()

Enter a product name/keyword : Apple Mac Book M2 Starlight
Searching for the product on Amazon...
Search on Amazon completed.
Extracting the price from Amazon...
Price extraction from Amazon completed.
Searching for the product on Flipkart...
Search on Flipkart completed.
Extracting the price from Flipkart...
Price extraction from Flipkart completed.
Searching for the product on Croma...
Search on Croma completed.
Extracting the price from Croma...
Price extraction from Croma completed.


In [19]:
# Print the prices
# Remove non-numeric characters, including '₹' and commas, and convert to float
def extract_numeric(price_str):
    if price_str:
        return float(''.join(filter(str.isdigit, price_str)))
    else:
        return None

amazon_price_numeric = extract_numeric(amazon_price)
flipkart_price_numeric = extract_numeric(flipkart_price)
croma_price_numeric = extract_numeric(croma_price)

# Print the numeric values
print(f"Amazon Price: {amazon_price_numeric}")
print(f"Flipkart Price: {flipkart_price_numeric}")
print(f"Croma Price: {croma_price_numeric}")


Amazon Price: 107990.0
Flipkart Price: 107890.0
Croma Price: 107990.0


In [21]:
# Find the cheapest price and website
cheapest_price = min(filter(None, [amazon_price_numeric, flipkart_price_numeric, croma_price_numeric]))
cheapest_website = None

if amazon_price_numeric == cheapest_price:
    cheapest_website = "Amazon"
elif flipkart_price_numeric == cheapest_price:
    cheapest_website = "Flipkart"
elif croma_price_numeric == cheapest_price:
    cheapest_website = "Croma"

if cheapest_website:
    print(f"The best website to buy the product '{product_keyword}' is {cheapest_website} with a price of {cheapest_price}")
else:
    print(f"No price information available for the product '{product_keyword}' from any website.")

The best website to buy the product 'Apple Mac Book M2 Starlight' is Flipkart with a price of 107890.0
